# 搜索爬虫

---

In [ ]:
import re
import threading
from warnings import filterwarnings
from pandas import read_csv, DataFrame, merge
from time import sleep
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options

filterwarnings("ignore")

In [ ]:
def split_list(lst, num_splits):
    split_size = len(lst) // num_splits
    remainder = len(lst) % num_splits

    splits = []
    index = 0

    for _ in range(num_splits):
        split_end = index + split_size + (1 if remainder > 0 else 0)
        splits.append(lst[index:split_end])
        index = split_end
        remainder -= 1

    return splits


def collect_Baidu(company_names, extra, num):
    '''
    collecting.........
    '''
    global bai_du_result
    global bai_du_failed
    global task_length
    
    subtasks = split_list(company_names, num)
    max_subtasks = max([len(i) for i in subtasks])
    
    # start driver-------------------------------
    options = Options()
    options.headless = True
    driver = webdriver.Firefox(options=options)
    wait = WebDriverWait(driver, 10)
    driver.get('https://www.baidu.com/')
    
    # parameters
    tests = [0 for _ in range(num)]
    
    # open tabs
    for _ in range(num - 1):
        driver.execute_script("window.open('https://www.baidu.com/', '_blank');")
        
    for i in range(num):
        driver.switch_to.window(driver.window_handles[i])
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#kw')))
    
    # collecting
    for j in range(max_subtasks):
        
        for i in range(num):
            driver.switch_to.window(driver.window_handles[i])
            if j >= len(subtasks[i]):
                pass
            else:
                # search for the company j
                box = driver.find_element(By.CSS_SELECTOR, '#kw')
                box.clear()
                box.send_keys('\"' + subtasks[i][j] + '\"' + extra)
                box.send_keys(Keys.RETURN)

        for i in range(num):
            try:
                driver.switch_to.window(driver.window_handles[i])
                if j >= len(subtasks[i]):
                    pass
                else:
                    # collect data
                    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.hint_PIwZX')))
                    not_loaded = [True for _ in range(num)]
                    while not_loaded[i]:
                        test = bs(driver.page_source, 'html.parser').select_one('#content_left > div')
                        if test != tests[i]:
                            not_loaded[i] = False
                        sleep(1)
                    tests[i] = test
                    _result = bs(driver.page_source, 'html.parser').select_one('.hint_PIwZX').get_text()
                    _result = re.findall(r'\d{1,3}(?:,\d{3})*|\d+', _result)[0]
                    bai_du_result.append({'简称' : subtasks[i][j], '百度搜索结果' : _result})
                    with print_lock:
                        print(f"==========Finished:\033[1;32m{round((len(bai_du_result) / task_length) * 100, 3)}%\033[0m Failed:\033[1;31m{len(bai_du_failed)}\033[0m==========".ljust(40), end='\r')
            except:
                bai_du_failed.append(subtasks[i][j])
            
    driver.quit()

    return None


def collect_Google(company_names, extra, num):
    '''
    collecting.........
    '''
    global google_result
    global google_failed
    global task_length
    
    subtasks = split_list(company_names, num)
    max_subtasks = max([len(i) for i in subtasks])
    
    # start driver-------------------------------
    driver = webdriver.Firefox()
    wait = WebDriverWait(driver, 10)
    driver.get('https://www.google.com.hk/advanced_search?hl=zh-CN&fg=1')
    
    # parameters
    tests = [0 for _ in range(num)]
    
    # open tabs
    for _ in range(num - 1):
        driver.execute_script("window.open('https://www.google.com.hk/advanced_search?hl=zh-CN&fg=1', '_blank');")
        sleep(1)
        
    # search
    for i in range(num):
        driver.switch_to.window(driver.window_handles[i])
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#xX4UFf')))
        box = driver.find_element(By.CSS_SELECTOR, '#xX4UFf')
        box.clear()
        box.send_keys('京东')
        box.send_keys(Keys.RETURN)
        sleep(1)
        
    print('请通过各个网页的放爬虫机制')
    input('完成后输入任何文字来继续 ： ')
        
    for i in range(num):
        driver.switch_to.window(driver.window_handles[i])
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#APjFqb')))
    
    for j in range(max_subtasks):
        
        for i in range(num):
            driver.switch_to.window(driver.window_handles[i])
            if j >= len(subtasks[i]):
                pass
            else:
                # search for the company j
                box = driver.find_element(By.CSS_SELECTOR, '#APjFqb')
                box.clear()
                box.send_keys('\"' + subtasks[i][j] + '\"' + extra)
                box.send_keys(Keys.RETURN)
                sleep(1.5)

        for i in range(num):
            try:
                driver.switch_to.window(driver.window_handles[i])
                if j >= len(subtasks[i]):
                    pass
                else:
                    # collect data
                    try:
                        if '异常流量' in bs(driver.page_source, 'html.parser').select_one('body > div:nth-child(1) > div:nth-child(5)').get_text():
                            print('请通过各个网页的放爬虫机制')
                            input('完成后输入任何文字来继续 ： ')
                    except:
                        pass
                    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#result-stats')))
                    not_loaded = [True for _ in range(num)]
                    while not_loaded[i]:
                        test = bs(driver.page_source, 'html.parser').select_one('#center_col')
                        if test != tests[i]:
                            not_loaded[i] = False
                        sleep(1)
                    tests[i] = test
                    _result = bs(driver.page_source, 'html.parser').select_one('#result-stats').get_text()
                    _result = re.findall(r'\d{1,3}(?:,\d{3})*|\d+', _result)[0]
                    google_result.append({'简称' : subtasks[i][j], 'Google搜索结果' : _result})
                    with print_lock:
                        print(f"==========Finished:\033[1;32m{round((len(google_result) / task_length) * 100, 3)}%\033[0m Failed:\033[1;31m{len(google_failed)}\033[0m==========".ljust(40), end='\r')
            except:
                google_failed.append(subtasks[i][j])
            
    driver.quit()

    return None

## 参数定义，请输入你要加入的额外关键词

In [ ]:
data = read_csv('/home/nolan/Documents/Notebook/scraper/南通公司列表(带股东).csv')

# 这里输入额外关键词
extra = ' 官网'
# 这里输入你想设定的多线程数量，越多相当于开更多的浏览器
# 注意并不是反比例增长，越多的浏览器会带来越大的运行压力，降低所有浏览器运行速度
num_threads = 1
# 这里输入每个浏览器同时开多少个页面
num_tabs = 1
# 这里输入每个google浏览器同时开多少个页面
Gnum_tabs = 5

print_lock = threading.Lock()
task_length = data.shape[0]

## 百度

In [ ]:
threads = []
bai_du_result = []
bai_du_failed = []
tasks = split_list(data['简称'].to_list(), num_threads)

for _ in range(num_threads):
    thread = threading.Thread(target=collect_Baidu, args=(tasks[_], extra, num_tabs))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

In [ ]:
# 合并
data = merge(data, pd.Dataframe(bai_du_result), on='简称')

## Google

In [ ]:
google_result = []
google_failed = []
collect_Google(data['简称'].to_list(), extra, Gnum_tabs)

In [ ]:
# 展示前20个
google_result[:20]

In [ ]:
# 合并
data = merge(data, pd.Dataframe(google_result), on='简称')

## 保存

In [ ]:
data.to_csv('./(已搜索)全国专精特新小巨人_搜索验证.csv')